# Notebook 4 — Full Pipeline Demo

This notebook demonstrates the complete DocAuth analysis pipeline on a single document:

1. **Signature verification** — are two signatures from the same person?
2. **Copy-move detection** — has any region been duplicated within the document?
3. **ELA** — does the compression pattern suggest manipulation?
4. **OCR** — extract all text for downstream analysis
5. **Summary report** — combined verdict

In [ ]:
import sys
sys.path.insert(0, '..')

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image

SIG_DIR  = Path('../Signature Detection and Analysis/data/test/021')
DOC_PATH = Path('../Copy Move Forgery/CopyMoveDetection/test_images/cheque22.png')

genuine = sorted(SIG_DIR.glob('genuine-*.png'))
forged  = sorted(SIG_DIR.glob('forged-*.png'))
doc_img = Image.open(DOC_PATH).convert('RGB') if DOC_PATH.exists() else None
print('Ready:', DOC_PATH.name if DOC_PATH.exists() else 'No document found')

## Step 1 — Signature Verification

In [ ]:
weights = Path('../weights/siamese_best.pt')
if weights.exists() and len(genuine) >= 2 and len(forged) >= 1:
    from src.signature.inference import verify
    sig_genuine = verify(genuine[0], genuine[1], weights=weights)
    sig_forged  = verify(genuine[0], forged[0],  weights=weights)
    print('Genuine pair:', sig_genuine)
    print('Forged pair: ', sig_forged)
else:
    sig_genuine = {'verdict': 'N/A — train model first', 'confidence': 0.0}
    sig_forged  = {'verdict': 'N/A — train model first', 'confidence': 0.0}
    print('Skipping signature verification (no weights found).')

## Step 2 — Copy-Move Detection

In [ ]:
if DOC_PATH.exists():
    from src.copy_move.detector import detect_copy_move
    cm_result = detect_copy_move(DOC_PATH)
    print('Copy-Move:', cm_result['verdict'], f'(score={cm_result["score"]:.3f})')
else:
    cm_result = {'verdict': 'N/A', 'score': 0.0}
    print('No document image found.')

## Step 3 — ELA

In [ ]:
if doc_img is not None:
    from src.analysis.ela import generate_ela, ela_score
    ela_img   = generate_ela(doc_img, quality=95, scale=15)
    ela_score_val = ela_score(ela_img)
    ela_verdict = 'Forged' if ela_score_val > 0.08 else ('Suspicious' if ela_score_val > 0.03 else 'Authentic')
    print(f'ELA: {ela_verdict} (score={ela_score_val:.4f})')
else:
    ela_verdict = 'N/A'
    ela_score_val = 0.0

## Step 4 — OCR

In [ ]:
if DOC_PATH.exists():
    from src.analysis.ocr import extract_text
    ocr = extract_text(DOC_PATH)
    print('OCR text:', ocr['full_text'][:500])
else:
    ocr = {'full_text': 'N/A', 'avg_confidence': 0.0}

## Step 5 — Summary Report

In [ ]:
from datetime import datetime

print('=' * 55)
print('  DOCAUTH FORENSIC REPORT')
print(f'  Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
print('=' * 55)
print(f'  Document           : {DOC_PATH.name if DOC_PATH.exists() else "N/A"}')
print(f'  Signature (genuine): {sig_genuine["verdict"]} (conf={sig_genuine["confidence"]:.2%})')
print(f'  Signature (forged) : {sig_forged["verdict"]}  (conf={sig_forged["confidence"]:.2%})')
print(f'  Copy-Move          : {cm_result["verdict"]} (score={cm_result["score"]:.3f})')
print(f'  ELA                : {ela_verdict} (score={ela_score_val:.4f})')
print(f'  OCR confidence     : {ocr["avg_confidence"]:.1%}')
print('=' * 55)

## Visual Summary

In [ ]:
if doc_img is not None:
    from src.copy_move.visualizer import overlay_heatmap
    import numpy as np

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    axes[0].imshow(doc_img)
    axes[0].set_title('Original Document')
    axes[0].axis('off')

    axes[1].imshow(ela_img)
    axes[1].set_title(f'ELA Map ({ela_verdict})')
    axes[1].axis('off')

    if cm_result['mask'].any() if 'mask' in cm_result else False:
        overlay = overlay_heatmap(np.array(doc_img), cm_result['mask'])
        axes[2].imshow(overlay)
        axes[2].set_title(f'Copy-Move ({cm_result["verdict"]})')
    else:
        axes[2].imshow(doc_img)
        axes[2].set_title(f'Copy-Move: {cm_result["verdict"]}')
    axes[2].axis('off')

    plt.suptitle('DocAuth — Full Pipeline Analysis', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print('No document image to display.')